# Chapter 12: 고급 LoRA 기법

## 1. 학습 목표

* **DoRA (Weight-Decomposed LoRA)**와 **AdaLoRA** 등 고급 PEFT 기법의 원리를 이해한다.
* `use_dora=True` 등 간단한 설정으로 학습 성능을 높이는 방법을 실습한다.
* 데이터 규모에 따른 최적의 **Rank** 선택 가이드를 익힌다.
* **선택적 레이어 학습**을 통해 재앙적 망각(Catastrophic Forgetting)을 방지한다.

## 2. DoRA (Weight-Decomposed LoRA)

### 2.1 개념

DoRA는 2024년에 제안된 기법으로, 가중치를 **크기(Magnitude)**와 **방향(Direction)**으로 분해하여 학습한다. LoRA는 방향만 업데이트하는 경향이 있는데, DoRA는 크기 벡터를 별도로 학습하여 Full Fine-tuning에 더 가까운 학습 패턴을 보인다.

```text
W = m × (V / ||V||)
- m: 크기 벡터 (Magnitude) → 직접 학습 (파라미터 수 매우 적음)
- V: 방향 행렬 (Direction) → LoRA로 학습 (저순위 행렬 분해)

```

### 2.2 장점

* 같은 Rank에서 일반 LoRA보다 더 높은 성능을 보인다.
* 특히 낮은 Rank (r=4, 8)에서도 학습 안정성이 뛰어나다.
* 별도의 추론 오버헤드가 없다 (병합 시 일반 LoRA와 동일).

### 2.3 설정 실습

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 로드 (예시)
#model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-14B", ...)

# DoRA 설정
# use_dora=True 옵션만 추가하면 된다.
dora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True  # 핵심 설정: DoRA 활성화
)

print("DoRA 설정 완료: use_dora=True")
#model = get_peft_model(model, dora_config)

## 3. AdaLoRA (Adaptive LoRA)

### 3.1 개념

모든 레이어에 동일한 Rank를 할당하는 것은 비효율적일 수 있다. AdaLoRA는 학습 과정에서 **중요한 레이어에는 높은 Rank**를, 덜 중요한 레이어에는 낮은 Rank를 동적으로 할당한다. 이를 통해 파라미터 예산을 효율적으로 사용한다.

### 3.2 설정 실습

In [10]:
from peft import AdaLoraConfig

# AdaLoRA 설정
adalora_config = AdaLoraConfig(
    init_r=16,              # 초기 Rank (약간 높게 시작)
    target_r=8,             # 목표 평균 Rank (최종적으로 맞출 Rank)
    tinit=200,              # 워밍업 스텝 (초기에는 Rank 조절 안 함)
    tfinal=1000,            # Rank 조절 종료 스텝
    deltaT=10,              # Rank 업데이트 간격
    orth_reg_weight=0.1,    # 정직교성 규제 가중치
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    total_step=10000
)

print("AdaLoRA 설정 완료: 동적 Rank 할당")

AdaLoRA 설정 완료: 동적 Rank 할당


## 4. Rank 선택 가이드

Rank(`r`)는 학습 가능한 파라미터 수와 직결된다. 무조건 높다고 좋은 것은 아니며, 데이터 규모와 태스크 복잡도에 따라 적절한 값을 선택해야 한다.

### 4.1 데이터 규모별 권장 Rank

| 데이터 규모 (예제 수) | 권장 Rank | 설명 |
| --- | --- | --- |
| **500 ~ 1,000** | **r=8** | 적은 데이터로 과적합 방지, 표면적인 톤앤매너 학습에 적합 |
| **1,000 ~ 5,000** | **r=16** | 일반적인 SFT에 가장 권장되는 값, 안정적인 성능 |
| **5,000 ~ 20,000** | **r=32** | 복잡한 지시사항이나 새로운 지식 주입이 필요할 때 |
| **20,000+** | **r=64** | 데이터가 충분히 많고, 논리적 추론 능력을 강화해야 할 때 |

In [11]:
def recommend_rank(dataset_size):
    """데이터 크기에 따른 Rank 추천 함수"""
    if dataset_size < 1000:
        return 8
    elif dataset_size < 5000:
        return 16
    elif dataset_size < 20000:
        return 32
    else:
        return 64

# 예시 확인
dataset_sizes = [800, 3000, 15000, 50000]
print("데이터 규모별 Rank 추천:")
for size in dataset_sizes:
    print(f"  데이터 {size}개 → 권장 Rank: {recommend_rank(size)}")

데이터 규모별 Rank 추천:
  데이터 800개 → 권장 Rank: 8
  데이터 3000개 → 권장 Rank: 16
  데이터 15000개 → 권장 Rank: 32
  데이터 50000개 → 권장 Rank: 64


## 5. 선택적 레이어 학습 (Layer Selection)

모델의 **하위 레이어(앞쪽)**는 일반적인 언어적 특징(문법 등)을 담당하고, **상위 레이어(뒤쪽)**는 구체적인 정보나 태스크 특화 기능을 담당하는 경향이 있다.
재앙적 망각(Catastrophic Forgetting)을 막고 싶다면, 앞쪽 레이어는 보존하고 뒤쪽 레이어만 학습하는 전략이 유효하다.

In [12]:
# 후반부 레이어만 학습하도록 설정하는 예시
# gpt-oss-20b (Qwen-14B)는 약 40~48개의 레이어를 가짐
total_layers = 40
train_start_layer = 20  # 절반 이후부터 학습

selective_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],

    # layers_to_transform: 특정 인덱스의 레이어만 LoRA 적용
    layers_to_transform=list(range(train_start_layer, total_layers)),

    task_type="CAUSAL_LM"
)

print(f"선택적 학습 설정: Layer {train_start_layer} ~ {total_layers-1}만 학습")
print("효과: 초기 레이어의 일반 지식 보존, 후기 레이어의 태스크 적응")

선택적 학습 설정: Layer 20 ~ 39만 학습
효과: 초기 레이어의 일반 지식 보존, 후기 레이어의 태스크 적응


## 6. 요약

| 기법 | 특징 | 추천 상황 |
| --- | --- | --- |
| **Standard LoRA** | 가장 기본적이고 검증됨 | 일반적인 모든 상황 (Start here) |
| **DoRA** | 크기와 방향 분리 학습 | **성능 최적화**가 필요할 때 (강력 추천) |
| **AdaLoRA** | 레이어별 동적 Rank | 파라미터 효율을 극한으로 추구할 때 |
| **선택적 학습** | 특정 레이어만 타겟팅 | 기존 지식 유지(망각 방지)가 중요할 때 |

실무에서는 **DoRA (`use_dora=True`)**를 기본으로 사용하고, 데이터 양에 맞춰 Rank를 16~32 사이로 조절하는 것이 가장 무난하고 강력한 조합이다.

다음 챕터는 **Chapter 13: 모델 평가 및 테스트**로, 학습된 모델의 성능을 객관적인 지표(Perplexity, BLEU 등)와 LLM-as-Judge 방식으로 평가하는 방법을 다룬다.